# Databases
Guest lecture by [Bradley Dice](https://github.com/bdice/).

### Why databases?
Tons of applications: managing digital content (websites), product inventories, transactions, health records... structure, security, and speed are all important.

- What if the size of your data exceeds the memory of the system?
  - Out-of-core algorithms.
- What if your data needs to be accessed by servers around the world?
  - Split it up by geography.
- Text files and spreadsheets can only do so much.

## What can we expect from most databases? CRUD + ACID.

Operations you can perform: **CRUD.** When interfacing with a database, you can expect to have these functions available.
- **Create:** Add new data.
- **Read:** Fetch existing data.
- **Update:** Edit existing data.
- **Delete:** Remove data.

Guarantees for how the database will act: **ACID.** These properties ensure databases are protected against hardware failures and software errors.
- **Atomicity:** Transactions will be completed fully, or not at all.
- **Consistency:** Transactions must leave the database in a valid state.
- **Isolation:** Transactions performed concurrently must act the same as transactions performed sequentially.
- **Durability:** Transactions must be permanent (i.e. stored on a disk, not in memory).

### Types of databases
- Relational and non-relational.
- These generally correspond to "SQL-like" and "NoSQL-like"

## Relational Databases
Programmers interface with relational databases through Structured Query Language (SQL).
### Examples:
Open Source:
- MySQL
- PostgreSQL (aka Postgres)
- SQLite (for local, application-level storage)

Commercial:
- Oracle Database (aka Oracle)
- Microsoft SQL Server
- DB2 (IBM)
- Microsoft Access, though casual users might not even realize it

## Non-relational Databases
Sometimes the structure of relational databases can be limiting. What if some properties are defined for one object but not another? There are a few types of non-relational databases including *column stores* (e.g. Cassandra), *document stores* (e.g. MongoDB), *key-value stores* (e.g. Redis), and *graph databases* (e.g. Neo4j).

### Examples:
Open Source:
- MongoDB
- Apache Cassandra
- Redis
- Apache CouchDB
- Neo4j

## Python Example with sqlite3

In [1]:
import sqlite3
import os

database = 'test.sqlite'

# Removes the database if it already exists
if os.path.exists(database):
    os.remove(database)

**Create** data with the `CREATE TABLE` and `INSERT` statements.

In [2]:
with sqlite3.connect(database) as conn:
    cursor = conn.cursor()
    cursor.execute(
        """CREATE TABLE people
        (first_name text, last_name text, job text, age real)""")
    cursor.execute(
        """INSERT INTO people VALUES (?,?,?,?)""",
        ('Bradley', 'Dice', 'Guest Lecturer', 25))

We can also just use `conn` directly, with a shorter syntax that creates cursors automatically. Also, we use `executemany` with a list of tuples, to perform many actions at once.

In [3]:
# Let's add lots of people
famous_actors = [
    ('Robert', 'Downey Jr.', 'Iron Man', 53),
    ('Chris', 'Evans', 'Captain America', 37),
    ('Scarlett', 'Johansson', 'Black Widow', 33),
    ('Samuel', 'Jackson', 'Nick Fury', 69),
    ('Benedict', 'Cumberbatch', 'Dr. Strange', 42),
    ('Brie', 'Larson', 'Captain Marvel', 29),
    ('Chadwick', 'Boseman', 'Black Panther', 40),
]
# Yes, I'm a Marvel fan
with sqlite3.connect(database) as conn:
    conn.executemany("""INSERT INTO people VALUES (?,?,?,?)""", famous_actors)

**Read** data with the `SELECT` statement.

In [4]:
with sqlite3.connect(database) as conn:
    cursor = conn.cursor()
    cursor.execute(
        """SELECT * FROM people""")
    print(cursor.fetchall())

[('Bradley', 'Dice', 'Guest Lecturer', 25.0), ('Robert', 'Downey Jr.', 'Iron Man', 53.0), ('Chris', 'Evans', 'Captain America', 37.0), ('Scarlett', 'Johansson', 'Black Widow', 33.0), ('Samuel', 'Jackson', 'Nick Fury', 69.0), ('Benedict', 'Cumberbatch', 'Dr. Strange', 42.0), ('Brie', 'Larson', 'Captain Marvel', 29.0), ('Chadwick', 'Boseman', 'Black Panther', 40.0)]


**Update** data with the `UPDATE` statement. We forgot a middle initial! Here we use `WHERE` to identify rows that match a set of criteria.

In [5]:
with sqlite3.connect(database) as conn:
    conn.execute(
        """UPDATE people SET first_name = ? WHERE first_name = ? AND last_name = ?""",
        ("Samuel L.", "Samuel", "Jackson"))

We can return specific columns by listing them after `SELECT`, and filter the rows with `WHERE`.

In [6]:
with sqlite3.connect(database) as conn:
    results = conn.execute("""SELECT first_name, last_name FROM people WHERE last_name = ?""", ("Jackson",))
    print(list(results))

[('Samuel L.', 'Jackson')]


SQL also lets you make complex selections, groupings, and filterings. Here is an example.

In [7]:
with sqlite3.connect(database) as conn:
    results = conn.execute("""SELECT COUNT(*) FROM people""")
    print('Total count of people:', list(results))
    results = conn.execute(
        """SELECT SUBSTR(first_name, 1, 1) AS first_letter, COUNT(*) FROM people GROUP BY first_letter""")
    print('Count by first letters of first names:', list(results))
    results = conn.execute("""SELECT AVG(age) FROM people""")
    print('Average age of people:', list(results))
    results = conn.execute("""SELECT SUM(age) FROM people""")
    print('Summed ages of people:', list(results))

Total count of people: [(8,)]
Count by first letters of first names: [('B', 3), ('C', 2), ('R', 1), ('S', 2)]
Average age of people: [(41.0,)]
Summed ages of people: [(328.0,)]


There are tons of functions for math, string manipulations, date/time manipulations, and more. If you want to learn to do something in particular, just Google it. StackExchange is a great place to find SQL examples. The full reference manual for MySQL is here: https://dev.mysql.com/doc/refman/8.0/en/

In [8]:
# Show us the guts of the database! This command is SQLite-specific.
with sqlite3.connect(database) as conn:
    results = conn.execute("""SELECT * FROM sqlite_master WHERE type = 'table'""")
    print(list(results))

[('table', 'people', 'people', 2, 'CREATE TABLE people\n        (first_name text, last_name text, job text, age real)')]


The last operation is **delete**. Here's how we delete data, also called "dropping" rows/columns/tables.

In [9]:
with sqlite3.connect(database) as conn:
    conn.execute("""DELETE FROM people WHERE first_name = ?""", ("Bradley",))
    results = conn.execute("""SELECT COUNT(*) FROM people""")
    print('Total count of people after removing Bradley:', list(results))
    # You can't rename or remove columns in sqlite, but this is how you would do it in most SQL databases:
    #conn.execute("""ALTER TABLE people DROP COLUMN age""")
    conn.execute("""DROP TABLE people""")
    print('The table "people" has been dropped.')

Total count of people after removing Bradley: [(7,)]
The table "people" has been dropped.


### ...so what does it mean to be "relational"?
Now we're going to dive into some of the most important parts of SQL: keys and relationships. Think about your UMich ID card. It gives you a unique identifying number that isn't your name (there could be many Jane Smiths). It is a permanent number and won't ever change, and nobody else will ever have the same ID number. Similarly, most databases need a **primary key**.

In [10]:
# Demos of relational data

### Sources

- https://www.alooma.com/blog/types-of-modern-databases
- https://en.wikipedia.org/wiki/Outline_of_databases
- https://www.jamesserra.com/archive/2015/08/relational-databases-vs-non-relational-databases/
- https://www.sqlite.org/whentouse.html
- https://en.wikipedia.org/wiki/NoSQL
- https://docs.python.org/3.7/library/sqlite3.html